In [1]:
# prompt: Create a code for model zero shot, few shot, and chain of thought prompting of quick running LLM and show consistency issues.

!pip install transformers

from transformers import pipeline

# Load a simple, quick-running model
# distilgpt2 is a good choice for quick inference
generator = pipeline('text-generation', model='gpt2') # LLAMA-2-2B, #Gemma,
# GPT-2, GPT-J, GPT-Neo, ....

def zero_shot_prompt(prompt_text):
  """Zero-shot prompting: Give the model a prompt without examples."""
  response = generator(prompt_text, max_length=10, num_return_sequences=1)[0]['generated_text']
  return response.strip()

def few_shot_prompt(prompt_text, examples):
  """Few-shot prompting: Give the model a prompt with a few examples."""
  formatted_prompt = "\n".join([f"{example['input']}\n{example['output']}" for example in examples])
  formatted_prompt += f"\n{prompt_text}"
  response = generator(formatted_prompt, max_length=10, num_return_sequences=1)[0]['generated_text']
  #num_return_sequences = 5 ; ">1" --> Beam Search, Top-p (nucleus sampling), Top-k
  # max_length =1 to max_length = 1000
  # We need to try and extract the part after the final input
  try:
      output_start = response.rfind(prompt_text) + len(prompt_text)
      return response[output_start:].strip()
  except:
      return response.strip() # Fallback if extraction fails

def chain_of_thought_prompt(prompt_text):
  """Chain-of-thought prompting: Ask the model to think step-by-step."""
  cot_prompt = f"{prompt_text} Let's think step by step."
  response = generator(cot_prompt, max_length=10, num_return_sequences=1)[0]['generated_text']
  return response.strip()





/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [2]:
# Example Task: Classify the sentiment of a sentence as Positive or Negative.

input_sentence = "This movie was bad!"

print("--- Zero-Shot Prompting ---")
zero_shot_response = zero_shot_prompt(f"Classify the sentiment of the following sentence: '{input_sentence}' Sentiment:")
print(f"Input: '{input_sentence}'")
print(f"Response: {zero_shot_response}")
print("-" * 25)


print("--- Few-Shot Prompting ---")
# sentiment = Neutral
# Definition -- What is Positive Sentiment? What is negative sentiment? What is neutral sentiment? (User is providing)
# Let the model generate a definition in the few-shot and Chain of Thought scenario.
# Extend the number of few-shot
few_shot_examples = [
    {"input": "Classify the sentiment of the following sentence: 'The service was good.' Sentiment:", "output": "positive"},
    {"input": "Classify the sentiment of the following sentence: 'I hate the food.' Sentiment:", "output": "Negative"}
]
few_shot_response = few_shot_prompt(f"Classify the sentiment of the following sentence: '{input_sentence}' Sentiment:", few_shot_examples)
print(f"Input: '{input_sentence}'")
print(f"Response: {few_shot_response}")
print("-" * 25)

print("--- Chain-of-Thought Prompting ---")
cot_response = chain_of_thought_prompt(f"Classify the sentiment of the following sentence: '{input_sentence}' Sentiment:")
print(f"Input: '{input_sentence}'")
print(f"Response: {cot_response}")
print("-" * 25)


# Demonstrate consistency issues with a different input
input_sentence_2 = "The weather is just okay."

print("\n--- Demonstrating Consistency Issues ---")

print("\n--- Zero-Shot Prompting (Second Input) ---")
zero_shot_response_2 = zero_shot_prompt(f"Classify the sentiment of the following sentence: '{input_sentence_2}' Sentiment:")
print(f"Input: '{input_sentence_2}'")
print(f"Response: {zero_shot_response_2}")
print("-" * 25)

print("\n--- Few-Shot Prompting (Second Input) ---")
few_shot_response_2 = few_shot_prompt(f"Classify the sentiment of the following sentence: '{input_sentence_2}' Sentiment:", few_shot_examples)
print(f"Input: '{input_sentence_2}'")
print(f"Response: {few_shot_response_2}")
print("-" * 25)

print("\n--- Chain-of-Thought Prompting (Second Input) ---")
cot_response_2 = chain_of_thought_prompt(f"Classify the sentiment of the following sentence: '{input_sentence_2}' Sentiment:")
print(f"Input: '{input_sentence_2}'")
print(f"Response: {cot_response_2}")
print("-" * 25)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


--- Zero-Shot Prompting ---


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Input: 'This movie was bad!'
Response: Classify the sentiment of the following sentence: 'This movie was bad!' Sentiment:I'm not saying that the Clintons should be president,' but rather that they should be president of the United States.' That is a statement that 'we're doing our best to maintain the same message of hope and hope," the Clintons said in a statement released by the Bill and Melinda Clinton Foundation on Saturday.

That statement, which was released by the foundation, was in response to the recent revelations that in a videotaped interview, Hillary Clinton was asked about her use of a private email server while she was secretary of state, which was found to have been compromised by the Russians.

The Clinton campaign on Sunday accused the foundation of "slandering" its supporters, saying it was "trying to spin" the Clinton Foundation's statements into a political ploy to "lose the election" and "get away with it."

The foundation, along with the Clinton Foundation, is ru

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

Input: 'This movie was bad!'
Response: Classify the sentiment of the following sentence: 'This movie was bad!' Sentiment: Let's think step by step.He is not a coward, a coward who is going to get caught with his hands dirty.' I said, 'No, I am not. I am not. I am not. I am not.' His reply was, 'It is a great pity that you do not know.' He was a coward who went to the jail, and was caught up in the scandal. He made a great mistake, he was not a coward, he was a coward who went to jail, and was caught up in the scandal. He was a man who was going to become a man himself. This is an interesting thing. I really think that your opinion is not a good one. I think that your opinion is what you are. You cannot go on with the past, you cannot go on with the future, you cannot go on with the dream. I think that your opinion is a very good one. I think that your opinion is not a good one at all. I think that you should not read too much into the past, that your opinion is one of a bad one. I thin

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Input: 'The weather is just okay.'
Response: Classify the sentiment of the following sentence: 'The weather is just okay.' Sentiment:I can't believe we're even talking about this anymore, but I do, because I've seen it all before.'

That is a simple statement, but it is also very much a false one. The word "temptation" is a noun. It is used only in a very specific way, to achieve a sort of self-loathing. "Temptation" is a word for "temptation" — to fear — but not to love. It means to "fight back" — to "make an effort" — to make an effort to be happy. "Temptation" is not a term for "to make an effort." It is a word for "to give up" — a desperate act. It is a word for "to give up" — a desperate act.

The word has been used in almost every form of speech for centuries. But because of the nature of the word, the meaning of the word has become a word for "temptation," not a word for "temptation." The word itself is not a word for "temptation." The word itself is not a word for "temptation."

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Input: 'The weather is just okay.'
Response: The Republican Party is going to have a very special place in our hearts and mind when it comes to electing a president who is beholden to the financial interests of the wealthiest 1 percent.'

He then added: 'When it comes to the economy, we need to look at the big picture and look at an American economy.'

Mr Obama's statement comes at a time when the Democrats have been struggling to win over more support for his campaign.

The White House said on Sunday that it will hold a press conference on Sunday to press Mr Obama to release the tax returns of his five wealthiest tax-exempt donors.

But Mr Obama has been under fire for his remarks in recent days, with former Secretary of State Hillary Clinton saying on Saturday that he was 'off the hook' for his remarks.

And on Saturday, the president said he had 'no idea' who he is 'on the hook' for his comments.

He also said that he had a 'great relationship' with former President Bill Clinton but